<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain_huggingface langchain_community  gradio transformers elasticsearch gradio_calendar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 

In [2]:
import os
from getpass import getpass
import requests
import os
from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from huggingface_hub import InferenceClient
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import json
import gradio as gr
from gradio_calendar import Calendar

def extract_primary_icd10(diagnosis_str):
    try:
        diagnoses = json.loads(diagnosis_str)
        if isinstance(diagnoses, list):
            for d in diagnoses:
                if isinstance(d, dict) and d.get("dx_type") == "PP":
                    icd10 = d.get("icd10")
                    if icd10:
                        # Remove the dot from the ICD-10 code
                        return icd10.replace(".", "")
    except (json.JSONDecodeError, TypeError):
        pass
    return None


In [3]:
# Elasticsearch credentials
user = "natthaphol.po"
passwords = "cnEM5CeFrG"
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
es = Elasticsearch(
    hosts=[{'host': 'elk.manageai.co.th', 'port': 443, 'scheme': 'https'}],
    basic_auth=(user, passwords)
)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def rag_encode(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.squeeze().tolist()

def search_exact_match_score(query_text, index_name):
    query_vector = rag_encode(query_text)
    search_query = {
        "size": 10,
        "_source": ["icd10", "total_average_price"],  # Include only required fields
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        exact_score_matches = [
            {
                "_source": hit["_source"]
            }
            for hit in response['hits']['hits'] if hit["_score"] == 2.0
        ]

        if exact_score_matches:
            return exact_score_matches
        else:
            print("No documents found with score 2.0.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None

def search_similar_vectors(query_text, index_name):
    query_vector = rag_encode(query_text)

    search_query = {
        "size": 1000,
        "_source": [
            "simb_billing_code", "local_billing_name", "item_id", "item_name",
            "average_initial_price", "order_count", "icd10", "range"
        ],
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        results = []
        for hit in response['hits']['hits']:
            score = hit["_score"]
            if score == 2.0:
                result = {
                    "_index": hit["_index"],
                    "_id": hit["_id"],
                    "_score": score,
                    "_source": hit["_source"]
                }
                results.append(result)

        if results:
            source_data = [hit["_source"] for hit in results]
            df = pd.DataFrame(source_data)
            return df
        else:
            print("No results with score 2.0 found.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

## gradio

In [4]:
# กำหนดตัวแปร Global สำหรับเก็บข้อมูลที่อัปโหลด
uploaded_data = None
uploaded_simb = None
uploaded_diag = None
sample_cids = []
excluded_sample_cids = []
wait_sample_cids = []
diagnosis_mapping = {}
# Mappings for illness type and follow-up status
illness_type_mapping = {
    "ILL": "เจ็บป่วย",
    "ACC": "อุบัติเหตุ",
    "FU": "ผู้ป่วยนอกติดตามอาการหลังจาก Admit เจ็บป่วย",
    "ER": "อุบัติเหตุฉุกเฉินภายใน 24 ชม.",
    "HD": "ผู้ป่วยนอกล้างไต",
    "DEN": "ทันตกรรม"
}

follow_up_mapping = {
    "N": "เข้ารับการรักษาครั้งแรก",
    "Y": "เข้ารักษาแบบต่อเนื่อง"
}

excluded_conditions = ["เนื้องอก", "มะเร็ง", "ต้อ", "ทอนซิล", "ไทรอยด์เป็นพิษ", "ซึมเศร้า"]

def filter_excluded_patients(patient_data, mapping, conditions):
    # Create excluded ICD-10 mapping based on the conditions
    excluded_icd10 = {}
    for code, description in mapping.items():
        if any(condition in description for condition in conditions):
            excluded_icd10[code] = description

    # Filter patients with the excluded diagnoses
    filtered_patients = []
    for _, row in patient_data.iterrows():
        try:
            # Parse diagnosis into a list of dictionaries
            diagnosis_list = json.loads(row["diagnosis"]) if isinstance(row["diagnosis"], str) else row["diagnosis"]
            for diag in diagnosis_list:
                if diag["icd10"] in excluded_icd10:
                    filtered_patients.append(row)
                    break
        except (TypeError, KeyError, json.JSONDecodeError):
            continue

    return pd.DataFrame(filtered_patients)

wait_disease = ['K300', 'K297', 'I495']
def filter_wait_disease_patients(patient_data, mapping, conditions):
    wait_icd10 = {}
    for code, description in mapping.items():
        if any(condition in description for condition in conditions):
            wait_icd10[code] = description

    filtered_patients = []
    for _, row in patient_data.iterrows():
        try:
            diagnosis_list = json.loads(row["diagnosis"]) if isinstance(row["diagnosis"], str) else row["diagnosis"]
            for diag in diagnosis_list:
                if diag["icd10"] in wait_icd10:
                    filtered_patients.append(row)
                    break
        except (TypeError, KeyError, json.JSONDecodeError):
            continue

    return pd.DataFrame(filtered_patients)


def load_files(data_file, simb_file, diag_file):
    global uploaded_data, uploaded_simb, uploaded_diag, sample_cids, diagnosis_mapping, excluded_sample_cids, wait_sample_cids

    try:
        if data_file:
            uploaded_data = pd.read_csv(data_file.name)
        if simb_file:
            uploaded_simb = pd.read_csv(simb_file.name)
        if diag_file:
            uploaded_diag = pd.read_csv(diag_file.name)
        if uploaded_diag is not None:
            diagnosis_mapping = {
                row['ICD-10 Code']: row['Description'].strip()
                for _, row in uploaded_diag.iterrows()
            }
        else:
            diagnosis_mapping = {}
        if uploaded_data is not None:
            sample_cids = uploaded_data['cid'].sample(2).tolist()
            print(f"ผู้ป่วยตัวอย่าง: {sample_cids}")
        else:
            sample_cids = []
        if uploaded_data is not None:
            excluded_patients = filter_excluded_patients(uploaded_data, diagnosis_mapping, excluded_conditions)
            excluded_sample_cids = excluded_patients['cid'].sample(2).tolist()
            print(f"ผู้ป่วยโรคร้าย: {excluded_sample_cids}")
        else:
            excluded_sample_cids = []
        if uploaded_data is not None:
            wait_patients = filter_wait_disease_patients(uploaded_data, diagnosis_mapping, wait_disease)
            wait_sample_cids = wait_patients['cid'].sample(2).tolist()
            print(f"(ผู้ป่วยโรคที่มีระยะเวลา 180 วัน: {wait_sample_cids}")
        else:
            wait_sample_cids = []

        return "ไฟล์ถูกอัปโหลดสำเร็จ!", gr.update(choices=sample_cids), gr.update(choices=excluded_sample_cids), gr.update(choices=wait_sample_cids), gr.update(choices=list(diagnosis_mapping.values()))

    except Exception as e:
        return f"เกิดข้อผิดพลาดในการอัปโหลดไฟล์: {str(e)}", gr.update(choices=[]), gr.update(choices=[]), gr.update(choices=[]), gr.update(choices=[])

# ฟังก์ชันสำหรับใช้งานไฟล์ที่อัปโหลด
def analyze_uploaded_data():
    global uploaded_data, uploaded_simb, uploaded_diag

    if uploaded_data is None or uploaded_simb is None or uploaded_diag is None:
        return "กรุณาอัปโหลดไฟล์ทั้งหมดก่อน", None

    # ตัวอย่างการประมวลผลข้อมูล (เช่น นับจำนวนแถวในแต่ละไฟล์)
    data_summary = {
        "data.csv": len(uploaded_data),
        "simblist.csv": len(uploaded_simb),
        "diaglist.csv": len(uploaded_diag),
    }
    result_message = "ประมวลผลไฟล์สำเร็จ!"
    return result_message, data_summary

def update_diagnosis_choices(cid):
    global diagnosis_mapping
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        icd10_code = patient_row['icd10'].iloc[0]
        diagnosis_description = diagnosis_mapping.get(icd10_code, "")
        return diagnosis_description
    return ""

def populate_patient_info(cid):
    global uploaded_data, uploaded_diag
    # ค้นหาผู้ป่วยตาม CID
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        patient_data = json.loads(patient_row.iloc[0]["patient"])
        icd10_code = patient_row.iloc[0]["icd10"]
        diagnosis_description = uploaded_diag[uploaded_diag['ICD-10 Code'] == icd10_code]['Description'].iloc[0] if not uploaded_diag[uploaded_diag['ICD-10 Code'] == icd10_code].empty else ""
        billing_details = patient_row.iloc[0]["billing"]
        visit_date = pd.to_datetime(patient_row.iloc[0]["created_at"]).strftime('%Y-%m-%d')

        illness_type_code = patient_row["illnes_type"].iloc[0]
        illness_type_mapped = illness_type_mapping.get(illness_type_code, "อุบัติเหตุ")
        visit_type_mapped = follow_up_mapping.get(patient_row["is_follow_up"].iloc[0], "Unknown")
        chief_complaint = patient_row["chief_complaint"].iloc[0]
        billing_table, total_net = get_billing_table_with_order_items(cid)

        return (
            patient_data.get("title_name", ""),
            patient_data.get("first_name", ""),
            patient_data.get("last_name", ""),
            patient_data.get("hn", ""),
            patient_data.get("dob", ""),
            visit_date,
            illness_type_mapped,
            visit_type_mapped,
            chief_complaint,
            diagnosis_description.strip(),
            billing_table.values.tolist(),
            billing_details,
            total_net,
            cid
        )
    else:
        return "", "", "", "", "", "", "", "", "", [], [], "", "", ""

def update_diagnosis_choices(cid):
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        icd10_code = patient_row['icd10'].iloc[0]
        diagnosis_description = diagnosis_mapping.get(icd10_code, "")
        return diagnosis_description  # Set as single choice and default value
    return ""

def get_simb_code(bill_description):
    description_to_simb = dict(zip(simb['Description'], simb['SIMB_1']))
    return description_to_simb.get(bill_description, "")

def get_billing_table_with_order_items(cid):
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        billing_data = json.loads(patient_row.iloc[0]["billing"])
        order_items_data = json.loads(patient_row.iloc[0]["order_item"])

        billing_df = pd.DataFrame(billing_data)
        order_items_df = pd.DataFrame(order_items_data)

        all_rows = []

        for _, billing_row in billing_df.iterrows():
            matching_items = order_items_df[order_items_df['simb_billing_code'] == billing_row['simb_billing_code']]
            all_rows.append({
                "ลำดับ": len(all_rows) + 1,
                "หมวดหมู่ค่าบริการ": f"{billing_row['simb_billing_code']} {billing_row['local_billing_name']}",
                "รายการ": "",
                "จำนวน": billing_row.get('item_amount', ""),
                "จำนวนเงิน(ก่อนหักส่วนลด)": float(billing_row['billing_initial']),
                "ส่วนลด": float(billing_row['billing_discount']),
                "จำนวนเงิน(หลังหักส่วนลด)": float(billing_row['billing_net_amount']),
                "Billing Code": "",
                "SIMB": billing_row['simb_billing_code']
            })

            for _, item in matching_items.iterrows():
                all_rows.append({
                    "ลำดับ": "",
                    "หมวดหมู่ค่าบริการ": item['item_id'],
                    "รายการ": item['item_name'],
                    "จำนวน": item['item_amount'],
                    "จำนวนเงิน(ก่อนหักส่วนลด)": float(item['initial']),
                    "ส่วนลด": float(item['discount']),
                    "จำนวนเงิน(หลังหักส่วนลด)": float(item['net_amount']),
                    "Billing Code": billing_row['local_billing_code'],
                    "SIMB": billing_row['simb_billing_code']
                })

        total_initial = billing_df['billing_initial'].astype(float).sum()
        total_discount = billing_df['billing_discount'].astype(float).sum()
        total_net = billing_df['billing_net_amount'].astype(float).sum()

        all_rows.append({
            "ลำดับ": "",
            "หมวดหมู่ค่าบริการ": "รวม",
            "รายการ": "",
            "จำนวน": "",
            "จำนวนเงิน(ก่อนหักส่วนลด)": total_initial,
            "ส่วนลด": total_discount,
            "จำนวนเงิน(หลังหักส่วนลด)": total_net,
            "Billing Code": "",
            "SIMB": ""
        })

        # Grand total row
        all_rows.append({
            "ลำดับ": "",
            "หมวดหมู่ค่าบริการ": "รวมค่ารักษาพยาบาลทั้งหมด",
            "รายการ": "",
            "จำนวน": "",
            "จำนวนเงิน(ก่อนหักส่วนลด)": total_initial,
            "ส่วนลด": total_discount,
            "จำนวนเงิน(หลังหักส่วนลด)": total_net,
            "Billing Code": "",
            "SIMB": ""
        })

        return pd.DataFrame(all_rows), total_net  # Return both DataFrame and total_net

    return pd.DataFrame(columns=["ลำดับ", "หมวดหมู่ค่าบริการ", "รายการ", "จำนวน", "จำนวนเงิน(ก่อนหักส่วนลด)", "ส่วนลด", "จำนวนเงิน(หลังหักส่วนลด)", "Billing Code", "SIMB"]), 0.0

def update_billing_table_with_order_items(cid):
    billing_df, total_net = get_billing_table_with_order_items(cid)
    return billing_df.values.tolist(), total_net

def handle_form(id_card, first_name, last_name, patient_id, birth_date, visit_date, treatment_type, visit_type,
                chief_complaint, diag_type, diagnosis, total_net, billing_details, policy_start_date, policy_end_date,
                service_type, remaining_opd_year, insurance_detail
):
    prompt_template = f"""
    ### Patient Information
    - ID Card: {id_card or 'N/A'}
    - Name: {first_name or 'N/A'} {last_name or 'N/A'}
    - Patient ID (HN): {patient_id or 'N/A'}
    - Date of Birth: {birth_date or 'N/A'}
    - Visit Date: {visit_date or 'N/A'}
    - Treatment Type: {treatment_type or 'N/A'}
    - Visit Type: {visit_type or 'N/A'}
    - Chief Complaint: {chief_complaint or 'N/A'}
    - Diagnosis Type: {diag_type or 'N/A'}
    - Diagnosis (ICD10): {diagnosis or 'N/A'}

    ### Billing Information
    - Total Bill Amount (Net): {total_net or '0.00'} THB
    - Billing Details: {billing_details or 'N/A'}

    ### Policy Information
    - Policy Start Date: {policy_start_date or 'N/A'}
    - Policy End Date: {policy_end_date or 'N/A'}
    - Service Type: {service_type or 'N/A'}
    - Remaining OPD Year: {remaining_opd_year or 'N/A'}
    - Insurance Detail: {insurance_detail or 'N/A'}
    """

    return prompt_template


In [5]:
custom_css = """
.red-border {
    border: 2px solid red !important;
    border-radius: 5px;
    padding: 5px;
}
"""
### Ui
with gr.Blocks() as demo:
    gr.Markdown("## SmartClaim ฝั่งประกัน 🛡️")
    with gr.Tab("Upload Data Files"):
        gr.Markdown("### อัปโหลดไฟล์ CSV ที่ใช้ในการวิเคราะห์")

        # Input for CSV files
        with gr.Row():
            data_file = gr.File(label="อัปโหลดไฟล์ data.csv")
            simb_file = gr.File(label="อัปโหลดไฟล์ simblist.csv")
            diag_file = gr.File(label="อัปโหลดไฟล์ diaglist.csv")

        # Display message after uploading files
        upload_button = gr.Button("อัปโหลดไฟล์")
        upload_result = gr.Textbox(label="ผลลัพธ์การอัปโหลดไฟล์", interactive=False)

    with gr.Tab("Patient and Claim Information"):
        gr.Markdown("## ข้อมูลผู้ป่วยนอก")
        id_card = gr.Textbox(label="เลขบัตรประชาชน *", placeholder="กรอกเลขบัตรประชาชนของผู้ป่วย", elem_classes="red-border")

        with gr.Row():
            sample_cid_buttons = gr.Radio(label="เลือกตัวอย่าง CID", choices=[], type="value")
            sample_excluded_cid_buttons = gr.Radio(label="คนไข้เป็นโรคร้ายแรง / ประกันไม่คุ้มครอง", choices=[], type="value")
            sample_wait_cid_buttons = gr.Radio(label="คนไข้เป็นโรคที่มีระยะเวลา 180 วัน", choices=[], type="value")

        with gr.Row():
            title = gr.Dropdown(
                label="คำนำหน้า",
                choices=["คุณ", "นาง", "นางสาว", "นาย", "Mr", "Ms", "Miss", "Mrs", "เด็กชาย", "เด็กหญิง"]
            )
            first_name = gr.Textbox(label="ชื่อ")
            last_name = gr.Textbox(label="นามสกุล")
        with gr.Row():
            patient_id = gr.Textbox(label="รหัสผู้ป่วย (HN)")
            birth_date = Calendar(label="วัน/เดือน/ปีเกิด (ค.ศ.)")
            visit_date = Calendar(label="วันที่เข้ารับการรักษา")

        gr.Markdown("### ข้อมูลส่งเคลม")
        with gr.Row():
            treatment_type = gr.Dropdown(
                label="ประเภทการรักษา",
                value="",  # ค่าที่ได้จากการแมพ
                choices=list(illness_type_mapping.values())
)
            visit_type = gr.Radio(
                label="ประเภทการเข้ารักษา",
                choices=list(follow_up_mapping.values())
            )
        with gr.Row():
            chief_complaint = gr.Textbox(label="อาการสำคัญที่เข้ามาโรงพยาบาล", value="")
            total_net = gr.Textbox(label="รวมค่ารักษาพยาบาลทั้งหมด", value="", interactive=True)

        gr.Markdown("#### Diagnosis Type")
        with gr.Row():
            diag_type = gr.Dropdown(
                label="Diagnosis Type",
                choices=["โรคหลักที่ให้การรักษา (PP)", "โรคประจำตัว (UD)", "การวินิจฉัยร่วม (CM)", "โรคแทรก (CP)", "โรคอื่นๆ (OT)"],
                value='โรคหลักที่ให้การรักษา (PP)'
            )
            diagnosis = gr.Dropdown(
                label="Diagnosis (ICD10) *",
                choices=list(diagnosis_mapping.values()) if diagnosis_mapping else [],
                value=""
            )

        gr.Markdown("#### รายการค่ารักษาพยาบาล")
        billing_table = gr.DataFrame(
            headers=["ลำดับ", "หมวดหมู่ค่าบริการ", "รายการ", "จำนวน", "จำนวนเงิน(ก่อนหักส่วนลด)", "ส่วนลด", "จำนวนเงิน(หลังหักส่วนลด)", "Billing Code", "SIMB"],
            interactive=True
        )
        billing_details = gr.Textbox(label="รายละเอียดรายการค่ารักษาพยาบาล", interactive=False, visible=False)

    with gr.Tab("Insurance Policy Detail"):
        with gr.Row():
            policy_start_date = Calendar(label="วันเวลาที่กรมธรรม์มีผลบังคับ", value="")
            policy_end_date = Calendar(label="วันเวลาที่สิ้นสุดความคุ้มครอง", value="")
        with gr.Row():
            service_type = gr.Dropdown(label="ประเภทประกัน", value="ประกันสุขภาพ", choices=['ประกันอุบัติเหตุ','ประกันสุขภาพ','ประกันโรคร้ายแรง'])
            remaining_opd_year = gr.Textbox(label="จำนวนเงิน OPD คงเหลือ *", placeholder="กรอกจำนวนเงิน OPD คงเหลือ", value=0)
        insurance_detail = gr.Textbox(label="ข้อมูลกรมธรรม์", placeholder="กรอกรายละเอียดความคุ้มครองของประกัน",)

        submit_button = gr.Button("Submit")

        Output = gr.Textbox(label="ข้อมูลสำหรับวิเคราะห์", interactive=False)

        id_card.change(
            fn=populate_patient_info,
            inputs=[id_card],
            outputs=[title, first_name, last_name, patient_id, birth_date, visit_date, treatment_type, visit_type, chief_complaint, diagnosis, billing_table, billing_details, total_net, id_card]
        )

        id_card.change(
            fn=update_diagnosis_choices,
            inputs=[id_card],
            outputs=[diagnosis]
        )

        sample_cid_buttons.change(
            fn=populate_patient_info,
            inputs=[sample_cid_buttons],
            outputs=[title, first_name, last_name, patient_id, birth_date, visit_date, treatment_type, visit_type, chief_complaint, diagnosis, billing_table, billing_details, total_net, id_card]
        )

        sample_excluded_cid_buttons.change(
            fn=populate_patient_info,
            inputs=[sample_excluded_cid_buttons],
            outputs=[title, first_name, last_name, patient_id, birth_date, visit_date, treatment_type, visit_type, chief_complaint, diagnosis, billing_table, billing_details, total_net, id_card]
        )

        sample_wait_cid_buttons.change(
            fn=populate_patient_info,
            inputs=[sample_wait_cid_buttons],
            outputs=[title, first_name, last_name, patient_id, birth_date, visit_date, treatment_type, visit_type, chief_complaint, diagnosis, billing_table, billing_details, total_net, id_card]
        )

        upload_button.click(
            fn=load_files,
            inputs=[data_file, simb_file, diag_file],
            outputs=[upload_result, sample_cid_buttons, sample_excluded_cid_buttons, sample_wait_cid_buttons, diagnosis]
        )

        submit_button.click(
            fn=handle_form,
            inputs=[id_card, first_name, last_name, patient_id, birth_date, visit_date, treatment_type, visit_type,
                    chief_complaint, diag_type, diagnosis, total_net, billing_details, policy_start_date, policy_end_date,
                    service_type, remaining_opd_year, insurance_detail],
            outputs=[Output]
        )

/usr/local/lib/python3.10/dist-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include:  or set allow_custom_value=True.
  warnings.warn(


In [6]:
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://acd3c1ec08d7c69a6f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ผู้ป่วยตัวอย่าง: ['1219900514439', '1101402178198']
ผู้ป่วยโรคร้าย: ['1129900520693', '1770600018468']
(ผู้ป่วยโรคที่มีระยะเวลา 180 วัน: ['1101700660907', '3720500042801']
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://acd3c1ec08d7c69a6f.gradio.live


 # **condition**

In [113]:
#case1: โรคร้ายแรง
excluded_conditions = ["เนื้องอก", "มะเร็ง", "ต้อ", "ทอนซิล", "ไทรอยด์เป็นพิษ","ซึมเศร้า"]
#case2: โรคที่มีระยะเวลา 180 วัน
wait_disease = ['K300', 'K297', 'I495']

### prompt

In [ ]:
import requests
import json

# Your credentials for API access
username = "manageai2024"
passwords = "ManageAI@2024"
auth = (username, passwords)

# Define the API URL
url = 'https://ai-api.manageai.co.th/llm-model-03/'

# Prepare model parameters (you can adjust these parameters)
model_param = {
    "max_new_tokens": 500,
    "temperature": 0.7,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

# Define the input data
input_data = ""

# Replace {input_data} in the prompt with the actual input data
formatted_prompt = prompt.replace("{input_data}", input_data)

# Send the request to the API
response = requests.post(
    url=url,
    data=json.dumps({"inputs": formatted_prompt, "parameters": model_param}),
    headers={'Content-Type': 'application/json'},
    auth=auth
)

print(response.json().get('generated_text', 'No generated text found'))